In [1]:
# import os
# # 
# # Install java
# ! apt-get update -qq
# ! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
# ! java -version

# # Install pyspark
# ! pip install --ignore-installed pyspark==2.4.4

# # Install Spark NLP
# ! pip install --ignore-installed spark-nlp

In [2]:
import sparknlp
import os
# 
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version; ", spark.version)

Spark NLP version:  2.5.5
Apache Spark version;  2.4.5


In [25]:
# downloading news dataset

In [3]:
!wget -O news_category_train.csv https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/news_Category/news_category_train.csv

--2020-10-06 21:17:53--  https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/news_Category/news_category_train.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.77.70
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.77.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24032125 (23M) [text/csv]
Saving to: 'news_category_train.csv'

     0K .......... .......... .......... .......... ..........  0%  116K 3m21s
    50K .......... .......... .......... .......... ..........  0%  317K 2m17s
   100K .......... .......... .......... .......... ..........  0%  172K 2m16s
   150K .......... .......... .......... .......... ..........  0% 10.4M 1m43s
   200K .......... .......... .......... .......... ..........  1%  175K 1m48s
   250K .......... .......... .......... .......... ..........  1% 9.80M 91s
   300K .......... .......... .......... .......... ..........  1% 10.8M 78s
   350K .......... .......... .......... 

  8400K .......... .......... .......... .......... .......... 36% 11.5M 6s
  8450K .......... .......... .......... .......... .......... 36% 9.18M 6s
  8500K .......... .......... .......... .......... .......... 36% 11.5M 6s
  8550K .......... .......... .......... .......... .......... 36% 11.8M 6s
  8600K .......... .......... .......... .......... .......... 36% 11.8M 6s
  8650K .......... .......... .......... .......... .......... 37% 10.8M 6s
  8700K .......... .......... .......... .......... .......... 37% 11.6M 6s
  8750K .......... .......... .......... .......... .......... 37% 11.9M 6s
  8800K .......... .......... .......... .......... .......... 37% 11.9M 6s
  8850K .......... .......... .......... .......... .......... 37% 9.02M 6s
  8900K .......... .......... .......... .......... .......... 38% 11.9M 6s
  8950K .......... .......... .......... .......... .......... 38% 11.5M 6s
  9000K .......... .......... .......... .......... .......... 38% 11.8M 6s
  9050K ....

In [4]:
!wget -O news_category_test.csv https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/news_Category/news_category_test.csv


 21500K .......... .......... .......... .......... .......... 91% 54.0M 0s
 21550K .......... .......... .......... .......... .......... 92% 56.8M 0s
 21600K .......... .......... .......... .......... .......... 92% 67.6M 0s
 21650K .......... .......... .......... .......... .......... 92% 48.6M 0s
 21700K .......... .......... .......... .......... .......... 92% 47.0M 0s
 21750K .......... .......... .......... .......... .......... 92% 53.4M 0s
 21800K .......... .......... .......... .......... .......... 93% 51.5M 0s
 21850K .......... .......... .......... .......... .......... 93% 49.9M 0s
 21900K .......... .......... .......... .......... .......... 93% 57.3M 0s
 21950K .......... .......... .......... .......... .......... 93% 47.4M 0s
 22000K .......... .......... .......... .......... .......... 93% 45.3M 0s
 22050K .......... .......... .......... .......... .......... 94% 39.4M 0s
 22100K .......... .......... .......... .......... .......... 94% 48.9M 0s
 22150K ...

In [6]:
# !head news_category_train.csv

The content is inside `description` column and the labels are inside `category` column

In [7]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("news_category_train.csv")

In [8]:
trainDataset.show()

+--------+--------------------+
|category|         description|
+--------+--------------------+
|Business| Short sellers, W...|
|Business| Private investme...|
|Business| Soaring crude pr...|
|Business| Authorities have...|
|Business| Tearaway world o...|
|Business| Stocks ended sli...|
|Business| Assets of the na...|
|Business| Retail sales bou...|
|Business|" After earning a...|
|Business| Short sellers, W...|
|Business| Soaring crude pr...|
|Business| OPEC can do noth...|
|Business| Non OPEC oil exp...|
|Business| WASHINGTON/NEW Y...|
|Business| The dollar tumbl...|
|Business|If you think you ...|
|Business|The purchasing po...|
|Business|There is little c...|
|Business|The US trade defi...|
|Business|Oil giant Shell c...|
+--------+--------------------+
only showing top 20 rows



In [9]:
trainDataset.count()

120000

In [10]:
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

In [11]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained() \
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("category")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [12]:
pipelineModel = pipeline.fit(trainDataset)

In [27]:
# !cd ~/annotator_logs && ls -l

In [26]:
# !cat ~/annotator_logs/ClassifierDLApproach_d82e68030034.log

#### (optional) use only when you want to save your pipeline



## Save and load pre-trained ClassifierDL pipeline

In [29]:
## if you're running in colab , it is not recommended to run this cell as colab has memory limitations
## run if your running locally

# pipelineModel.save("./classifierdl_pipeline")
# loadedPipeline = PipelineModel.load("./classifierdl_pipeline")
# loadedPipeline.transform(YOUR_DATAFRAME)

In [13]:
# dbfs:/ or hdfs:/ if you are saving it on distributed file systems
pipelineModel.stages[-1].write().overwrite().save('./tmp_classifierDL_model')



Let's use our pre-trained ClassifierDLModel in a pipeline: 

In [14]:

document = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained() \
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

classsifierdl = ClassifierDLModel.load("./tmp_classifierDL_model") \
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")

pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


Now let's load it back so we can have prediction all together with everything in that pipeline:

In [15]:
from pyspark.sql.types import StringType

dfTest = spark.createDataFrame([
    "Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.",
    "Scientists have discovered irregular lumps beneath the icy surface of Jupiter's largest moon, Ganymede. These irregular masses may be rock formations, supported by Ganymede's icy shell for billions of years..."
], StringType()).toDF("description")

In [16]:
prediction = pipeline.fit(dfTest).transform(dfTest)

In [17]:
prediction.select("class.result").show()

prediction.select("class.metadata").show(truncate=False)

+----------+
|    result|
+----------+
|[Business]|
|[Sci/Tech]|
+----------+

+---------------------------------------------------------------------------------------------------------------+
|metadata                                                                                                       |
+---------------------------------------------------------------------------------------------------------------+
|[[Sports -> 1.5947037E-7, Business -> 0.9999995, World -> 2.539525E-7, Sci/Tech -> 5.558146E-8, sentence -> 0]]|
|[[Sports -> 1.6789394E-14, Business -> 2.526997E-13, World -> 4.1692964E-12, Sci/Tech -> 1.0, sentence -> 0]]  |
+---------------------------------------------------------------------------------------------------------------+



# Evaluation 

Let's evaluatte our ClassifierDL model we trained earlier, saved it, and loaded it into a new pipeline by using a test dataset that model has never seen:

In [18]:
testDataset = spark.read \
      .option("header", True) \
      .csv("news_category_test.csv")

In [19]:
preds = pipelineModel.transform(testDataset)

In [20]:
preds.select('category','description',"class.result").show(50, truncate=50)

+--------+--------------------------------------------------+----------+
|category|                                       description|    result|
+--------+--------------------------------------------------+----------+
|Business|Unions representing workers at Turner   Newall ...|[Business]|
|Sci/Tech| TORONTO, Canada    A second team of rocketeers...|[Sci/Tech]|
|Sci/Tech| A company founded by a chemistry researcher at...|[Sci/Tech]|
|Sci/Tech| It's barely dawn when Mike Fitzpatrick starts ...|[Sci/Tech]|
|Sci/Tech| Southern California's smog fighting agency wen...|[Sci/Tech]|
|Sci/Tech|"The British Department for Education and Skill...|[Sci/Tech]|
|Sci/Tech|"confessed author of the Netsky and Sasser viru...|[Sci/Tech]|
|Sci/Tech|\\FOAF/LOAF  and bloom filters have a lot of in...|[Sci/Tech]|
|Sci/Tech|"Wiltshire Police warns about ""phishing"" afte...|[Sci/Tech]|
|Sci/Tech|In its first two years, the UK's dedicated card...|[Sci/Tech]|
|Sci/Tech| A group of technology companies  includi

In [21]:
preds_df = preds.select('category','description',"class.result").toPandas()

In [22]:
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

In [28]:
from sklearn.metrics import classification_report

Let's use `classification_report` from `sklearn` to evaluate the final scores. (keep in mind due to limited resources on a free Google Colab we only used 5 Epochs :)

In [24]:
print (classification_report(preds_df['result'], preds_df['category']))

              precision    recall  f1-score   support

    Business       0.86      0.83      0.85      1959
    Sci/Tech       0.88      0.86      0.87      1933
      Sports       0.98      0.95      0.97      1948
       World       0.86      0.93      0.89      1760

    accuracy                           0.89      7600
   macro avg       0.89      0.89      0.89      7600
weighted avg       0.89      0.89      0.89      7600

